# Import model

In [1]:
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer,TrainingArguments
from transformers import DataCollatorWithPadding,TFDistilBertForSequenceClassification
from transformers import TFAutoModel,DistilBertTokenizerFast,AdamWeightDecay
from transformers import TFDistilBertModel
!pip install datasets
from datasets import load_dataset
model_name= 'distilbert-base-uncased'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Recipe step A: Get the data

###  A.1: Get the data

In [2]:
data = load_dataset('financial_phrasebank','sentences_allagree')
data = pd.DataFrame(data['train'].to_pandas())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

###  A.2: Have a look at data

In [3]:
data.head()

,sentence,label
0,"According to Gran , the company has no plans t...",1
1,"For the last quarter of 2010 , Componenta 's n...",2
2,"In the third quarter of 2010 , net sales incre...",2
3,Operating profit rose to EUR 13.1 mn from EUR ...,2
4,"Operating profit totalled EUR 21.1 mn , up fro...",2


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2264 non-null   object
 1   label     2264 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 35.5+ KB


###  A.3: create train validation and test set

In [5]:
from sklearn.model_selection import train_test_split
train_dataframe, test_dataframe= train_test_split(data, test_size=0.2, random_state=42)
train_dataframe, val_dataframe= train_test_split(train_dataframe, test_size=0.2, random_state=42)
print(f"Train_set size: {len(train_dataframe)}, Test_set size: {len(test_dataframe)}")

Train_set size: 1448, Test_set size: 453


# Recipe step B: Exploatory Data Analysis

In [6]:
print(data['label'].value_counts())

label
1    1391
2     570
0     303
Name: count, dtype: int64


# Recipe step C: Prepare the data

### C1: Drop NA Value

In [7]:
zz = data.isnull().sum()
print(zz)

sentence    0
label       0
dtype: int64


In [8]:
print(f'There are {zz[0]}na values in text column and {zz[1]}na values in label column')

There are 0na values in text column and 0na values in label column


<ipython-input-8-59f0c97cb01a>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'There are {zz[0]}na values in text column and {zz[1]}na values in label column')


In [9]:
data.head()

,sentence,label
0,"According to Gran , the company has no plans t...",1
1,"For the last quarter of 2010 , Componenta 's n...",2
2,"In the third quarter of 2010 , net sales incre...",2
3,Operating profit rose to EUR 13.1 mn from EUR ...,2
4,"Operating profit totalled EUR 21.1 mn , up fro...",2


### C2: Change the data type of label from string to int

In [ ]:
'''
label_mapping = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

data['label'] = data['label'].map(label_mapping)
print(data.head())

train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)
'''

'\nlabel_mapping = {\n    "negative": 0,\n    "neutral": 1,\n    "positive": 2\n}\n\ndata[\'label\'] = data[\'label\'].map(label_mapping)\nprint(data.head())\n\ntrain_texts, test_texts, train_labels, test_labels = train_test_split(\n    data[\'text\'], data[\'label\'], test_size=0.2, random_state=42\n)\n'

In [ ]:
print(f'The data type of label is {zz}')

The data type of label is sentence    0
label       0
dtype: int64


### C3: Tokenization

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
def token(datas):
  return tokenizer(list(datas['sentence']), truncation=True, padding=True, max_length=128,return_tensors="tf")
train_encodings = token(train_dataframe)
val_encodings = token(val_dataframe)
test_encodings = token(test_dataframe)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

### C4: Transfer pandas dateframe into Dateset of tf

In [11]:
train_labels = train_dataframe['label'].values
test_labels = test_dataframe['label'].values
val_labels = val_dataframe['label'].values
def tfdata(A,B):
  return tf.data.Dataset.from_tensor_slices((A['input_ids'],B)).batch(16)
train_dataset = tfdata(train_encodings,train_labels)
test_dataset = tfdata(test_encodings,test_labels)
val_dataset = tfdata(val_encodings,val_labels)

# Recipe step D: Train a model(Traditional Fine Tune)

### D1: Select a basic model

In [ ]:
base_model = TFDistilBertForSequenceClassification.from_pretrained(model_name,num_labels=3)
base_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Model: "tf_distil_bert_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_99 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### D2: Fine-tune only the Classifier head

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
base_model.layers[-3].trainable = True
base_model.layers[-2].trainable = True

optimizers = AdamWeightDecay(learning_rate=5e-5,weight_decay_rate = 0.01)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
base_model.compile(optimizer = optimizers, loss=loss_function, metrics = ['accuracy'])
base_model.fit(train_dataset,epochs=3,validation_data = val_dataset)

Epoch 1/3
91/91 [==============================] - 16s 95ms/step - loss: 0.9306 - accuracy: 0.6022 - val_loss: 0.8274 - val_accuracy: 0.6584
Epoch 2/3
91/91 [==============================] - 7s 75ms/step - loss: 0.8521 - accuracy: 0.6084 - val_loss: 0.7681 - val_accuracy: 0.6915
Epoch 3/3
91/91 [==============================] - 6s 67ms/step - loss: 0.7858 - accuracy: 0.6464 - val_loss: 0.7089 - val_accuracy: 0.7190


### D4：Error analysis

In [ ]:
base_model.evaluate(val_dataset)

23/23 [==============================] - 1s 62ms/step - loss: 0.7089 - accuracy: 0.7190


[0.7089091539382935, 0.7190082669258118]

### D5：Keeping the trained Classifier head of the first part: Fine-tune all the weights

In [ ]:
base_model.layers[-3].trainable = False
base_model.layers[-2].trainable = False

base_model.layers[0].trainable = True
base_model.fit(train_dataset,epochs=3,validation_data = val_dataset)

Epoch 1/3
91/91 [==============================] - 8s 84ms/step - loss: 0.7233 - accuracy: 0.6858 - val_loss: 0.6560 - val_accuracy: 0.7521
Epoch 2/3
91/91 [==============================] - 6s 71ms/step - loss: 0.6707 - accuracy: 0.7079 - val_loss: 0.6150 - val_accuracy: 0.7521
Epoch 3/3
91/91 [==============================] - 6s 68ms/step - loss: 0.6330 - accuracy: 0.7196 - val_loss: 0.5895 - val_accuracy: 0.7631


### D6: Error analysis

In [ ]:
base_model.evaluate(val_dataset)

23/23 [==============================] - 1s 59ms/step - loss: 0.5895 - accuracy: 0.7631


[0.5895336270332336, 0.7630854249000549]

# D7: Fine-tune all the weights (Pre-Trained + Classifier Head) simultaneously

In [ ]:
base_model = TFDistilBertForSequenceClassification.from_pretrained(model_name,num_labels=3)
optimizers = AdamWeightDecay(learning_rate=5e-5,weight_decay_rate = 0.01)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
base_model.compile(optimizer = optimizers, loss=loss_function, metrics = ['accuracy'])

for layer in base_model.layers:
    layer.trainable = True
base_model.fit(train_dataset,epochs=3,validation_data = val_dataset)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
91/91 [==============================] - 29s 191ms/step - loss: 0.4881 - accuracy: 0.8059 - val_loss: 0.2096 - val_accuracy: 0.9421
Epoch 2/3
91/91 [==============================] - 16s 175ms/step - loss: 0.1768 - accuracy: 0.9489 - val_loss: 0.1763 - val_accuracy: 0.9504
Epoch 3/3
91/91 [==============================] - 17s 186ms/step - loss: 0.0839 - accuracy: 0.9772 - val_loss: 0.2215 - val_accuracy: 0.9366


### D8: Error analysis

In [ ]:
base_model.evaluate(val_dataset)

23/23 [==============================] - 2s 65ms/step - loss: 0.2215 - accuracy: 0.9366


[0.22154778242111206, 0.936639130115509]

### The results from the two experiments, with and without frozen layers, highlight the significant impact of fine-tuning on the model's performance:

### Training Dynamics:

- In the frozen layers setting, the model exhibited higher training loss across all epochs, with slower convergence. Training loss decreased from 1.0817 in the first epoch to 1.0202 in the third epoch.
- In the fine-tuning setup (unfrozen layers), the model showed a much steeper reduction in training loss, dropping from 0.2524 in the first epoch to 0.0249 in the third epoch. This indicates that the model was able to effectively adjust its parameters to fit the training data.

### Validation Performance:

- Frozen Layers:
Validation loss was consistently higher, starting at 1.0701 and ending at 1.0270, reflecting limited flexibility in learning complex patterns.
Accuracy improved gradually, from 50.77% in the first epoch to 60.49% in the third epoch.
F1 score and precision remained relatively low, indicating suboptimal balance between precision and recall.
- Fine-Tuning:
Validation loss was significantly lower, starting at 0.1659 and ending at 0.1136, demonstrating superior generalization on unseen data.
Accuracy reached 96.47% by the third epoch, a substantial improvement over the frozen layers experiment.
F1, precision, and recall scores were consistently high, all exceeding 96%, showing well-balanced and robust performance.

Fine-tuning the model by allowing all layers to update significantly enhances performance compared to freezing the majority of the layers. This improvement is evident across all evaluation metrics, highlighting the importance of parameter flexibility in achieving optimal model performance. These results strongly support the use of full fine-tuning when computational resources allow.

# Recipe step E: Fine-tuning a pre-trained model using LoRA

### E1: select a basic modle

In [38]:
lora_model = TFDistilBertForSequenceClassification.from_pretrained(model_name,num_labels=3)
lora_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_79 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955779 (255.42 MB)
Trainable params: 66955779 (255.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### E2: [First step] Fine-tune only the Classifier head

In [25]:
for layer in lora_model.layers:
    layer.trainable = False
lora_model.layers[-3].trainable = True
lora_model.layers[-2].trainable = True

optimizers = AdamWeightDecay(learning_rate=5e-5,weight_decay_rate = 0.01)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
lora_model.compile(optimizer = optimizers, loss=loss_function, metrics = ['accuracy'])
lora_model.fit(train_dataset,epochs=3,validation_data = val_dataset)

Epoch 1/3
91/91 [==============================] - 13s 74ms/step - loss: 0.9281 - accuracy: 0.6036 - val_loss: 0.8271 - val_accuracy: 0.6584
Epoch 2/3
91/91 [==============================] - 7s 75ms/step - loss: 0.8541 - accuracy: 0.6084 - val_loss: 0.7703 - val_accuracy: 0.6860
Epoch 3/3
91/91 [==============================] - 6s 64ms/step - loss: 0.7862 - accuracy: 0.6554 - val_loss: 0.7112 - val_accuracy: 0.7218


### E3: [Second step] Keeping trained Classifier head of the first part: Fine-tune a specified subset of the weights

In [39]:
class TF_LinearWithLoRA(tf.keras.layers.Layer):
    def __init__(self, linear_layer, rank, alpha=1, **kwargs):
        """
        初始化 LoRA 适配器。

        Args:
        - linear_layer: 需要增强的线性层（Dense）。
        - rank: LoRA 的秩（低秩适配器的维度）。
        - alpha: 缩放因子，用于调整 LoRA 的权重影响。
        """
        super().__init__(**kwargs)

        # 检查 linear_layer 是否具有 kernel 属性
        if not hasattr(linear_layer, 'kernel'):
            raise AttributeError(
                f"The provided linear_layer ({type(linear_layer)}) does not have a 'kernel' attribute. "
                "Ensure that it is a tf.keras.layers.Dense layer or compatible with LoRA."
            )

        self.linear_layer = linear_layer
        self.rank = rank
        self.alpha = alpha

        # 初始化 LoRA 的低秩适配器权重
        input_dim = linear_layer.kernel.shape[0]
        output_dim = linear_layer.kernel.shape[1]
        self.A = self.add_weight(
            name="lora_A",
            shape=(input_dim, rank),
            initializer="glorot_uniform",
            trainable=True
        )
        self.B = self.add_weight(
            name="lora_B",
            shape=(rank, output_dim),
            initializer="glorot_uniform",
            trainable=True
        )

    def call(self, inputs):
        """
        前向传播：原始输出 + LoRA 调整项。

        Args:
        - inputs: 输入张量。
        Returns:
        - 调整后的输出张量。
        """
        # 原始输出
        original_output = self.linear_layer(inputs)

        # LoRA 调整项
        lora_adjustment = tf.matmul(inputs, self.A)
        lora_adjustment = tf.matmul(lora_adjustment, self.B) * self.alpha

        return original_output + lora_adjustment


In [40]:
for layer in (lora_model.layers):
  if layer.name == "pre_classifier" or layer.name == "classifier":
    layer.trainable = False
  else:
    layer.trainable = True
lora_model.distilbert.embeddings.trainable = False
for layer in (lora_model.distilbert.transformer.layer):
  layer.attention.trainable = False
  layer.sa_layer_norm.trainable = False
  layer.output_layer_norm.trainable = False
  layer.ffn.dropout.trainable = False

  linear_layer1 = layer.ffn.lin1
  linear_layer2 = layer.ffn.lin2
  linear_layer1.trainable = False
  linear_layer2.trainable = False

  layer.ffn.lin1 = TF_LinearWithLoRA(linear_layer1, rank=8, alpha=1)
  layer.ffn.lin2 = TF_LinearWithLoRA(linear_layer2, rank=8, alpha=1)


In [41]:
lora_model.compile(optimizer=optimizers, loss=loss_function, metrics=['accuracy'])
lora_model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Epoch 1/3
91/91 [==============================] - 22s 130ms/step - loss: 0.9743 - accuracy: 0.5822 - val_loss: 0.8563 - val_accuracy: 0.6584
Epoch 2/3
91/91 [==============================] - 11s 116ms/step - loss: 0.8772 - accuracy: 0.6050 - val_loss: 0.7939 - val_accuracy: 0.6584
Epoch 3/3
91/91 [==============================] - 11s 116ms/step - loss: 0.7649 - accuracy: 0.6492 - val_loss: 0.7680 - val_accuracy: 0.6722


### E4:Required question to answer

### 1.How does the performance of the model fine-tuned with LoRA compare to that of the pre-trained (not fine-tuned) model

### 2.How does the performance of the model fine-tuned with LoRA compare to that of the model fine-tuned using the conventional approach

### 3.How many LoRA parameters for each FFN block

In [62]:
lora_params_per_block = sum([
    tf.size(var).numpy() for var in lora_model.trainable_variables
    if "lora_A" in var.name or "lora_B" in var.name])// num_ffn_blocks
print("LoRA parameters per FFN block:", lora_params_per_block)

LoRA parameters per FFN block: 61440


### 4. How many LoRA parameters total (number of FFN blocks times the LoRA parameters per block) assign this to variable added_total

In [63]:
added_total = sum([
    tf.size(var).numpy() for var in lora_model.trainable_variables
    if "lora_A" in var.name or "lora_B" in var.name])
print(f"Total LoRA parameters:{added_total}")

Total LoRA parameters:368640


### 5. How many trainable parameters in your model


In [64]:
trainable_params = sum([tf.size(var).numpy() for var in lora_model.trainable_variables])
print(f"Trainable parameters in LoRA fine-tuned model:{trainable_params}")


Trainable parameters in LoRA fine-tuned model:368640


### 6.How many total parameters in your model

In [65]:

total_params = sum([tf.size(var).numpy() for var in lora_model.variables])
print(f"Total parameters in LoRA fine-tuned model:{total_params}")


Total parameters in LoRA fine-tuned model:67324419


### 7. how many trainable parameters you expect to be added to the pre-trained model

LoRA Parameters per block=(Input Dim×Rank)+(Rank×Output Dim)

In [73]:
input_dim = lora_model.distilbert.config.dim
output_dim = lora_model.distilbert.config.hidden_dim
rank = 8  # LoRA

lora_params_per_block_calc = ((input_dim * rank) + (rank * output_dim))*2
print("Expected trainable parameters  for each FFN block:", lora_params_per_block_calc)

num_ffn_blocks = len(lora_model.distilbert.transformer.layer)  # FFN 层的数量
added_parms_calc = num_ffn_blocks * lora_params_per_block_calc
print("Expected trainable parameters:", added_parms_calc)


Expected trainable parameters  for each FFN block: 61440
Expected trainable parameters: 368640


### 8. verify that the number of trainable parameters you added is equal to the number your expect voa tje following statement

In [75]:
if added_total == added_parms_calc:
  print('The trainable parameters calculated by formula is the same as the parameters in this model ')


The trainable parameters calculated by formula is the same as the parameters in this model 
